In [1]:
import warnings
import pandas as pd
import numpy as np
from astropy.io.votable import parse_single_table, parse
from astropy.io.votable import from_table
import astropy.units as u
from astropy.coordinates import SkyCoord
from gevtev import gev_tev_data 


warnings.filterwarnings("ignore")

_epsilon = 5e-2

s = "data/simbad-plsr.xml"
votable1 = parse(s)
votable = parse_single_table(s)
table = votable.to_table()
table_plsr = table['RA_d','DEC_d', 'MAIN_ID']
pd_plsr = table_plsr.to_pandas() 
pd_plsr['class'] = 'plsr'

s = "data/simbad-be.xml"
votable1 = parse(s)
votable = parse_single_table(s)
table = votable.to_table()
table_be = table['RA_d','DEC_d', 'MAIN_ID']
pd_be = table_be.to_pandas()
pd_be['class'] = 'be'

frames_simbda = [pd_be, pd_plsr]
pd_simbad = pd.concat(frames_simbda)

In [2]:
common_data, only_tev_data, only_gev_data = gev_tev_data()

In [3]:
def common_gevtev_simbad(gevtev, simbad, epsilon):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a dictionary with corresponding values
    
    gevtev - a table of GeV and TeV 
    simbad - a table of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    glat_gevtev = gevtev['glat']
    glon_gevtev = gevtev['glon']
    pos_ra_simbad = pd_simbad.as_matrix(columns=['RA_d'])
    pos_dec_simbad = pd_simbad.as_matrix(columns=['DEC_d'])
    c_icrs = SkyCoord(ra=pos_ra_simbad*u.degree, dec=pos_dec_simbad*u.degree, frame='icrs')
    glat_simbad = c_icrs.galactic.b.deg
    glon_simbad = c_icrs.galactic.l.deg
    class_simbad = pd_simbad.as_matrix(columns=['class'])
    class_gevtev = gevtev['CLASS1']
    for i in range(len(glat_gevtev)):
        #if i % (len(glat_gevtev) // 4) == 0:
        #        print(i)
        for j in range(len(pos_ra_simbad)):
            
            #print(str(glat_gevtev[i]) + ' - ' + str(glat_simbad[j]))
            #print(str(glon_gevtev[i]) + ' - ' + str(glon_simbad[j]))
            if ((np.abs(glat_gevtev[i] - glat_simbad[j])/np.abs(glat_simbad[j]) < epsilon) 
                and (np.abs(glon_gevtev[i] - glon_simbad[j])/np.abs(glon_simbad[j]) < epsilon)) :
                    try: 
                        list(d.keys()).index(i)
                        d[i].append(j)
                        classes[str(i) + ' - ' + str(class_gevtev[i])].append(str(j) + ' - ' + str(class_simbad[j][0]))
                    except ValueError:
                        d.update({i : [j]})
                        classes.update({str(i) + ' - ' + str(class_gevtev[i]) : [str(j) + ' - ' + str(class_simbad[j][0])]})
    return d, classes

In [4]:
common_gevtev_simbad_, classes_ = common_gevtev_simbad(common_data, pd_simbad, _epsilon)
print(common_gevtev_simbad_)
print(classes_)

{3: [419], 6: [146]}
{'3 - PSR': ['419 - plsr'], '6 - PWN': ['146 - be']}


In [5]:
only_gev_simbad_, classes_ = common_gevtev_simbad(only_gev_data, pd_simbad, _epsilon)
print(only_gev_simbad_)
print(classes_)

{3: [909], 4: [485], 6: [628, 740], 7: [899], 10: [727], 12: [460, 909], 19: [70], 30: [460], 31: [71], 32: [899], 39: [477], 44: [727], 45: [333, 626, 911], 46: [232, 615, 627], 47: [485], 48: [477], 49: [628, 740], 56: [490], 57: [251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 316], 58: [477], 61: [899], 64: [251, 254, 259, 261, 264, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280], 66: [899], 73: [232, 627], 83: [701], 86: [139], 87: [455], 90: [139], 92: [461], 97: [333, 911], 103: [608], 104: [139], 105: [232], 108: [232, 614, 615, 627], 110: [475], 113: [608, 614, 615], 114: [59, 60, 356], 120: [710], 122: [251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 316], 123: [232, 627], 124: [475], 125: [310, 311, 312, 313, 314, 475], 127: [607, 608], 130: [139],

In [6]:
only_tev_simbad_, classes_ = common_gevtev_simbad(only_tev_data, pd_simbad, _epsilon)
print(only_tev_simbad_)
print(classes_)

{0: [70], 16: [487], 19: [560, 898], 22: [198, 200, 201, 202, 203, 204, 205, 206, 207, 210, 211, 213, 214, 215, 216, 315, 335, 342, 343, 344, 345, 346, 573, 575, 583, 586, 587, 588, 593, 685, 692, 693, 694, 695, 696, 697, 698, 699, 794, 795], 23: [198, 200, 201, 202, 203, 204, 205, 206, 210, 211, 213, 214, 315, 335, 344, 345, 346, 575, 583, 586, 587, 588, 593, 685, 692, 693, 694, 695, 696, 697, 698, 794, 795], 24: [198, 200, 201, 202, 203, 204, 210, 211, 213, 214, 315, 335, 344, 345, 346, 575, 583, 586, 587, 588, 593, 685, 692, 693, 694, 695, 696, 697, 698, 794, 795], 25: [415], 26: [89], 28: [106], 29: [875], 31: [736], 33: [783], 44: [496], 45: [227], 54: [507, 737], 72: [521, 529, 531, 905], 75: [305], 76: [305], 77: [934], 86: [340], 93: [153], 97: [148], 141: [725, 914]}
{'0 - pwn,snr': ['70 - be'], '16 - hbl': ['487 - plsr'], '19 - hbl': ['560 - plsr', '898 - plsr'], '22 - snr,mc': ['198 - be', '200 - be', '201 - be', '202 - be', '203 - be', '204 - be', '205 - be', '206 - be', '2

In [7]:

C = -1 * np.ones(len(pd_simbad))
for x in list(common_gevtev_simbad_.keys()):
        for y in common_gevtev_simbad_[x]:
            C[y] = x
pd_simbad['join'] = C
pd_common_gevtevsimbad = pd.merge(common_data, pd_simbad, left_index=True, right_on='join', how='inner')
del pd_common_gevtevsimbad['join']

pd_common_gevtevsimbad

,glat,glon,morph_pa,pos_ra,pos_dec,sed_dnde,sed_dnde_err,sed_e_ref,spec_dnde_1TeV,spec_dnde_1TeV_err,...,Flux1000_3000,Flux100_300,Flux3000_10000,Flux300_1000,Flux30_100,CLASS1,RA_d,DEC_d,MAIN_ID,class
72,4.265813,195.133850,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,...,5.716907e-07,1.782121e-06,1.170842e-07,1.374273e-06,NaN,PSR,96.986132,20.496320,b'* 16 Gem',plsr
146,-0.130943,332.365143,NaN,243.794891,-50.698036,"[6.08884e-11, 1.76839e-11, 4.3537e-12, 1.47045...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.378819, 0.667119, 1.21383, 2.12929, 3.80595...",6.655983e-12,4.635425e-13,...,1.036945e-08,1.936115e-07,2.540761e-09,5.640028e-08,NaN,PWN,258.575005,-38.983235,b'EM* AS 222',be
